In [3]:
# Installation de langdetect pour identifier les langues de notre df
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=d084bfdf28f33eebb077d92d847518e095baea2cb9fcf9ddf187a61e37a08825
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.3 MB/s eta 0:00:00


In [5]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import resample
from transformers import BertTokenizer, BertModel
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import wordnet
import nltk

from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
# Python libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import wordnet
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
import warnings

# Ignorer tous les avertissements
warnings.filterwarnings("ignore")

#python librairies

from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import sklearn.metrics as metrics


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Pa

In [6]:
from google.colab import drive
drive.mount('/content/drive')
path ="/content/drive/MyDrive/"
df1 = pd.read_csv(path + 'QueryResults.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:

# Séléction des colonnes d'intérets
df = df1[['Title', 'Body', 'Tags']]

# Détection de la langue
for index, row in df.iterrows():
    text = row['Body']
    try:
        lang = detect(text)
        df.loc[index, 'lang'] = lang
    except LangDetectException as e:
        df.loc[index, 'lang'] = 'unknown'

# Filtrage du df sur la langue la plus utilisée
df = df[df['lang']=='en']
df = df[['Body', 'Title','Tags']]

# Nettoyage des balises HTML et code
def remove_code(x):
    soup = BeautifulSoup(x, "lxml")
    code_to_remove = soup.findAll("code")
    for code in code_to_remove:
        code.replace_with(" ")
    return str(soup)

df['Body'] = df['Body'].apply(lambda text: remove_code(text))
df.Body = [BeautifulSoup(text,"lxml").get_text() for text in df.Body]

# Nettoyage du texte
def clean_text(text):
    if isinstance(text, list):
        text = ' '.join(text)
    text = text.lower()
    text = re.sub("\'\w+", ' ', text)
    text = text.encode("ascii", "ignore").decode()
    text = re.sub('[^\\w\\s#\\s++]', ' ', text)
    text = re.sub(r'\w*\d+\w*', ' ', text)
    text = re.sub('\s+', ' ', text)
    return text

df['Body'] = df['Body'].apply(lambda text: clean_text(text))
df['Title'] = df['Title'].apply(lambda text: clean_text(text))

# Nettoyage spécifique pour la colonne 'Tags'
df['Tags'] = df['Tags'].apply(lambda text: re.sub(r'<|>', ',', text))
df['Tags'] = df['Tags'].apply(lambda text: [tag for tag in text.split(',') if tag.strip()])
df['Tags'] = df['Tags'].apply(lambda text: ' '.join(text) if isinstance(text, list) else text)

# Tokenisation
def words_tokenize(text):
    text = text.split()
    return text

df['Body'] = df.Body.apply(lambda text: words_tokenize(text))
df['Title'] = df.Title.apply(lambda text: words_tokenize(text))
df['Tags'] = df.Tags.apply(lambda text: words_tokenize(text))

# Suppression des stopwords
stop_words = set(stopwords.words('english'))
df['Body_filtered'] = df['Body'].apply(lambda x: ' '.join([word for word in x if word.lower() not in stop_words]))
df['Title_filtered'] = df['Title'].apply(lambda x: ' '.join([word.lower() for word in x if word.lower() not in stop_words]))

df = df.drop(columns=['Body', 'Title'])
df = df.rename(columns={'Body_filtered': 'Body', 'Title_filtered': 'Title'})

# Lemmatisation
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
def lem_word(text):
    return [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in text]

df['Body'] = df.Body.apply(lem_word)
df['Title'] = df.Title.apply(lem_word)
df['Tags'] = df.Tags.apply(lem_word)

# Suppression des tags à occurrence unique
def remove_single_occurrence_tags(dataframe, column):
    all_tags = dataframe[column].explode()
    tag_counts = all_tags.value_counts()
    tags_to_remove = tag_counts[tag_counts == 1].index
    filtered_dataframe = dataframe[~dataframe[column].apply(lambda x: any(tag in tags_to_remove for tag in x))]
    return filtered_dataframe

filtered_df = remove_single_occurrence_tags(df, 'Tags')
df = filtered_df

# Your preprocessing function
def words_tokenize(text):
    text = text.split()  # Split into words
    return text

# Apply tokenization to create the 'Text' column
df['Text'] = df['Body'].apply(lambda x: ''.join(x)) + " " + df['Title'].apply(lambda x: ''.join(x))
df['Text'] = df.Text.apply(lambda text: ''.join(words_tokenize(text)))


In [43]:
# Importation du fichier test

df ='/content/drive/MyDrive/df_cleaned.csv'
df = pd.read_csv(df, index_col=False)

# Your preprocessing function
def words_tokenize(text):
    text = text.split()  # Split into words
    return text

# Apply tokenization to create the 'Text' column
df['Text'] = df['Body'].apply(lambda x: ''.join(x)) + " " + df['Title'].apply(lambda x: ''.join(x))
df['Text'] = df.Text.apply(lambda text: ''.join(words_tokenize(text)))

In [45]:
import pickle
import random
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import make_scorer, accuracy_score, recall_score, f1_score, average_precision_score, jaccard_score

# Initialize the "CountVectorizer" TFIDF for text
X = df["Text"]
vectorizer = TfidfVectorizer(analyzer="word",
                             max_df=0.97,
                             min_df=3,
                             tokenizer=None,
                             preprocessor=''.join,
                             stop_words=None,
                             lowercase=False)

# Fit and transform the data
X_tfidf = vectorizer.fit_transform(X)


# Multilabel binarizer for targets
y = df['Tags']

# Split tags and count their frequencies
tag_counts = {}
for tags in y:
    for tag in tags.split():
        tag_counts[tag] = tag_counts.get(tag, 0) + 1

# Select the top 100 most frequent tags
top_tags = sorted(tag_counts, key=tag_counts.get, reverse=True)[:100]

# Filter and transform tags using the selected top tags
tags_list_filtered = [[tag for tag in tags.split() if tag in top_tags] for tags in y]

# Initialize and fit the multilabel binarizer
multilabel_binarizer = MultiLabelBinarizer(classes=top_tags)
y = multilabel_binarizer.fit_transform(tags_list_filtered)

print("y shape : {}".format(y.shape))

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.25, random_state=42)

# Initialize and train the SGDClassifier using OneVsRestClassifier
sgd_classifier = OneVsRestClassifier(SGDClassifier())
sgd_classifier.fit(X_train, y_train)

# Predict the labels on the test set
y_pred = sgd_classifier.predict(X_test)

# Calculate the evaluation metrics
jaccard = jaccard_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)

# Print the evaluation metrics
print("Jaccard Score:", jaccard)
print("F1 Score:", f1)
print("Recall Score:", recall)
print("Accuracy Score:", accuracy)

y shape : (3453, 100)
Jaccard Score: 0.284998831163367
F1 Score: 0.4222138539923043
Recall Score: 0.33410351201478744
Accuracy Score: 0.13425925925925927


In [46]:
import joblib
import os

# Chemin d'enregistrement du modèle
path = "/content/drive/MyDrive/"
model_filename = "multilabel_binarizer.pkl"
model_path = os.path.join(path, model_filename)

# Enregistrer le modèle multilabel binarizer dans un fichier
joblib.dump(multilabel_binarizer, model_path)

# Chemin d'enregistrement du modèle pour le TF-IDF vectorizer
tfidf_vectorizer_filename = "tfidf_vectorizer.pkl"
tfidf_vectorizer_path = os.path.join(path, tfidf_vectorizer_filename)

# Enregistrer le TF-IDF vectorizer dans un fichier
joblib.dump(vectorizer, tfidf_vectorizer_path)

# Chemin d'enregistrement du modèle pour le SGD classifier
sgd_classifier_filename = "sgd_classifier.pkl"
sgd_classifier_path = os.path.join(path, sgd_classifier_filename)

# Enregistrer le SGD classifier dans un fichier
joblib.dump(sgd_classifier, sgd_classifier_path)

# Chemin d'enregistrement du modèle pour X_tfidf
x_tfidf_filename = "X_tfidf.pkl"
x_tfidf_path = os.path.join(path, x_tfidf_filename)

# Enregistrer X_tfidf dans un fichier
joblib.dump(X_tfidf, x_tfidf_path)

# Chemin d'enregistrement du modèle pour y_tag
y_tag_filename = "y_tag.pkl"
y_tag_path = os.path.join(path, y_tag_filename)

# Enregistrer y_tag dans un fichier
joblib.dump(y, y_tag_path)


['/content/drive/MyDrive/y_tag.pkl']

In [1]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00
  Created wheel for databricks-cli: filename=databricks_cli-0.17.7-py3-none-any.whl size=143855 sha256=4ccb60ebb452153a7c9cedd21badd54ce1f26a0557d67175a4d295922fb376f4
  Stored in directory: /root/.cache/pip/wheels/ae/63/93/5402c1a09c

In [2]:
pip install pydantic==1.10.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.2.1
    Uninstalling pydantic-2.2.1:
      Successfully uninstalled pydantic-2.2.1


In [24]:
import mlflow

In [47]:
import os
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

# Assuming you have already trained the SGD classifier and named it 'sgd_classifier'
final_model = sgd_classifier

# Define and fit the multilabel binarizer
multilabel_binarizer = MultiLabelBinarizer(classes=top_tags)
y = multilabel_binarizer.fit_transform(tags_list_filtered)

# Get the inferred signature for input and output data
signature = infer_signature(X_train, y)  # Use y_bin instead of y_train

# Chemin d'enregistrement du modèle avec le nom de dossier
path = "/content/drive/MyDrive/models"
model_folder = "final_model"

# Chemin complet pour enregistrer le modèle
model_path = os.path.join(path, model_folder)

# Enregistrer le modèle avec la signature
mlflow.sklearn.save_model(final_model, model_path, signature=signature)
